# plot and check Marchenko Pastur for different $\gamma=\frac{N}{M}$
To make sure I understood this correctly, I have to make sure everything works empirically. This tool a while with several AIs... Anyway, bottom line is that things seem to work. And for a certain $\gamma=\frac{N}{M}$ we shuold expect $s_{max}=1+
\sqrt{\gamma}$

In [212]:
import numpy as np
import matplotlib.pyplot as plt

# Parameters
N = 256
gamma_values = [0.5, 1, 2]
num_Ts = 20 

def generate_complex_matrix(N, M):
    """Generate a normalized complex Gaussian random matrix."""
    sigma = np.sqrt(2) / 2
    real_part = np.random.normal(0, sigma, size=(N, M))
    imag_part = np.random.normal(0, sigma, size=(N, M))
    A = (real_part + 1j * imag_part) / np.sqrt(M)  
    return A

def marchenko_pastur_svd_pdf(x, gamma):
    x_plus = (1 + np.sqrt(gamma)) ** 2
    x_minus = (1 - np.sqrt(gamma)) ** 2
    density = np.zeros_like(x)
    valid = (x ** 2 >= x_minus) & (x ** 2 <= x_plus)
    epsilon = 1e-15  # Small value to avoid division by zero
    density[valid] = (1 / (gamma*np.pi * np.maximum(x[valid], epsilon))) * np.sqrt((x_plus - x[valid] ** 2) * (x[valid] ** 2 - x_minus))
    
    if gamma > 1:
        density *= gamma
        
        # Add point mass at zero
        if x[0] == 0:
            density[0] = (gamma - 1) / gamma

    
    return density

fig, axs = plt.subplots(1, len(gamma_values), figsize=(18, 5))

for ax, gamma in zip(axs, gamma_values):
    M = int(N / gamma)
    singular_values = []
    
    # Generate matrices and compute singular values
    for _ in range(num_Ts):
        A = generate_complex_matrix(N, M)
        svd_vals = np.linalg.svd(A, compute_uv=False)
        singular_values.extend(svd_vals)
    
    singular_values = np.array(singular_values)
    print(f'{gamma=}')
    print(f'min={singular_values.min():.2f}, max={singular_values.max():.2f}')
    
    # Plot histogram of singular values
    ax.hist(singular_values, bins=50, density=True, alpha=0.7, label="Empirical")
    
    # Plot theoretical Marchenko-Pastur distribution for singular values
    x_vals = np.linspace(0, max(2, (1 + np.sqrt(gamma)) + 0.5), 1000)
    mp_pdf = marchenko_pastur_svd_pdf(x_vals, gamma)
    ax.plot(x_vals, mp_pdf, 'r-', label="Theoretical")
    
    min_sv = np.abs(1 - np.sqrt(gamma))
    max_sv = (1 + np.sqrt(gamma))
    
    ax.axvline(min_sv, color='g', linestyle='--', label=f'Min SV: {min_sv:.2f}')
    ax.axvline(max_sv, color='b', linestyle='--', label=f'Max SV: {max_sv:.2f}')
    
    if gamma > 1:
        ax.plot([0], [(gamma - 1) / gamma], 'ro', markersize=10, label="Point mass at 0")
    
    ax.set_title(f"γ = {gamma}")
    ax.set_xlabel("Singular Value")
    ax.set_ylabel("Density")
    ax.legend()
    fig.show()

plt.tight_layout()

gamma=0.5
min=0.29, max=1.71
gamma=1
min=0.00, max=2.00
gamma=2
min=0.41, max=2.41


In [108]:
N = 256
M = 512
A = generate_complex_matrix(N, M)
v_in = 1/np.sqrt(M)*np.ones(M)
(np.abs(A@v_in)**2).sum()

0.45002438108227266

# Expected maximal SVD value for finite size N 
This might explain the N dependance. For larger matrices we have a larger chance of hitting a large SVD value. (which is bounded from above by 4 for $N\rightarrow\infty$)

In [209]:
import numpy as np
from scipy.integrate import quad
import matplotlib.pyplot as plt

def marchenko_pastur_pdf(x, q):
    """
    PDF of the Marchenko-Pastur distribution.
    """
    a = (1 - np.sqrt(q))**2
    b = (1 + np.sqrt(q))**2
    if a <= x <= b:
        return np.sqrt((b - x) * (x - a)) / (2 * np.pi * q * x)
    else:
        return 0

def marchenko_pastur_cdf(x, q):
    """
    CDF of the Marchenko-Pastur distribution, computed by integrating the PDF.
    """
    a = (1 - np.sqrt(q))**2
    if x < a:
        return 0
    b = (1 + np.sqrt(q))**2
    if x > b:
        return 1
    result, _ = quad(lambda t: marchenko_pastur_pdf(t, q), a, x)
    return result

def expected_maximum(N, q):
    """
    Compute the expected maximum for N samples from the Marchenko-Pastur distribution.
    """
    a = (1 - np.sqrt(q))**2
    b = (1 + np.sqrt(q))**2

    def integrand(x):
        f_x = marchenko_pastur_pdf(x, q)
        F_x = marchenko_pastur_cdf(x, q)
        return x * N * (F_x**(N-1)) * f_x

    result, _ = quad(integrand, a, b)
    return result

# Parameters
q = 1  # Ratio M/N
N_values = 2**np.linspace(1, 12, 12)

# Compute E[max_s] for each N
expected_max_values = [expected_maximum(N, q) for N in N_values]

# Plot
fig, ax = plt.subplots(figsize=(8, 6))
ax.plot(N_values, expected_max_values, marker="o", label=f"Marchenko-Pastur (q={q})", color="blue")
# ax.set_xscale("log")
ax.set_xlabel("Number of Samples (N) [log scale]")
ax.set_ylabel("Expected Maximum (E[max_s])")
ax.set_title("Expected Maximum vs. Number of Samples for Marchenko-Pastur")
ax.grid(visible=True, which="both", linestyle="--", linewidth=0.5)
ax.legend()
fig.show()

# Print results for reference
for N, value in zip(N_values, expected_max_values):
    print(f"N={N}, E[max_s]={value:.6f}")


N=2.0, E[max_s]=1.540380
N=4.0, E[max_s]=2.130698
N=8.0, E[max_s]=2.669488
N=16.0, E[max_s]=3.096769
N=32.0, E[max_s]=3.405180
N=64.0, E[max_s]=3.615387
N=128.0, E[max_s]=3.753963
N=256.0, E[max_s]=3.843592
N=512.0, E[max_s]=3.900933
N=1024.0, E[max_s]=3.937388
N=2048.0, E[max_s]=3.960478
N=4096.0, E[max_s]=3.975073


# optimize SLM3 "analytically" with SVD on N//2 pixels  

## what does phase only do?
It seems thhat thie higest $s^2$ that I get for $N=256$ is $\approx1.95$, which is significantly less than $4$. And that having phase only reduces the energy transfer by $\approx0.7$.  I still get $1.6$ which is nice, but we need to cut this in half because we only half the matrix. We do get also 0.25 from the othe r half. TODO this. 

In [237]:
import numpy as np

N = 256
sig_for_gauss_iid = np.sqrt(2) / 2
T = 1 / np.sqrt(N) * np.random.normal(loc=0, scale=sig_for_gauss_iid, size=(N, N, 2)).view(np.complex128)[:, :, 0]

T_sub = T[N//2:, :N//2]
U, S, Vh = np.linalg.svd(T_sub)
max_energy_input = Vh[0].conj()  # Corresponding to the largest singular value
max_energy_input_phase_only = 1/np.sqrt(N/2)*np.ones_like(max_energy_input) * np.exp(1j*np.angle(max_energy_input))

max_energy_output = T_sub @ max_energy_input
max_energy_output_phase_only = T_sub @ max_energy_input_phase_only
energy = np.sum(np.abs(max_energy_output)**2)
energy_phase_only = np.sum(np.abs(max_energy_output_phase_only)**2)

input_energy = (np.abs(max_energy_input_phase_only)**2).sum()
print(f'{input_energy=}')
print("Maximized energy transfer:", energy)
print("Maximized energy transfer phase only:", energy_phase_only)
print("ratio:", energy_phase_only**2 / energy**2)


input_energy=1.0
Maximized energy transfer: 1.9094214194813701
Maximized energy transfer phase only: 1.5737479423464855
ratio: 0.6793080644973


In [55]:
# (np.abs(max_energy_input)**2).sum()
(np.abs(max_energy_input_phase_only)**2).sum()

0.5

In [245]:
import numpy as np

N = 256
sig_for_gauss_iid = np.sqrt(2) / 2
T = 1 / np.sqrt(N) * np.random.normal(loc=0, scale=sig_for_gauss_iid, size=(N, N, 2)).view(np.complex128)[:, :, 0]

# in_indexes = np.index_exp[N//2:] # TODO: choose randomly N/2 numbers, and extract the relevant sub matrix
T_sub = T[N//2:, :N//2]  
U, S, Vh = np.linalg.svd(T_sub)
good_input = Vh[0].conj()  # Corresponding to the largest singular value
tot_good_input_phase_only = 1/np.sqrt(N)*np.ones(N, dtype=np.complex128)
tot_good_input_phase_only[N//2:] *= np.exp(1j*np.angle(good_input))

output = T @ tot_good_input_phase_only
full_energy = np.sum(np.abs(output)**2)
half_energy = np.sum(np.abs(output[:N//2])**2)

output_sub = T_sub @ tot_good_input_phase_only[N//2:]
energy_sub = np.sum(np.abs(output_sub)**2)

print(f"{half_energy=:.2f}")
print(f"{full_energy=:.2f}")
print(f"{energy_sub=:.2f}")

half_energy=0.48
full_energy=0.90
energy_sub=0.81


# Tracy widom
which is a better approximation for the distribution for largest SVD values in large matrices

In [207]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
from scipy.linalg import svd

def largest_singular_value_distribution(N, num_samples=1000):
    sig_for_gauss_iid = np.sqrt(2) / 2
    largest_singular_values = np.zeros(num_samples)
    
    for i in range(num_samples):
        T = 1 / np.sqrt(N) * np.random.normal(loc=0, scale=sig_for_gauss_iid, size=(N, N, 2)).view(np.complex128)[:, :, 0]
        s = svd(T, compute_uv=False)
        largest_singular_values[i] = s[0]
    
    return largest_singular_values

N_values = 2**np.linspace(1, 8, 8, dtype=int)
x = np.linspace(-6, 6, 1000)

fig, ax = plt.subplots(figsize=(12, 8))

for N in N_values:
    # Simulate largest singular values
    largest_singular_values = largest_singular_value_distribution(N, num_samples=1000)
    print(f"N={N}, mean={(np.mean(largest_singular_values)**2)}")
    # Plot histogram of normalized singular values
    ax.hist(largest_singular_values, bins=50, density=True, alpha=0.5, label=f'N={N}')


ax.set_xlabel('Largest singular value')
ax.set_ylabel('Probability density')
ax.set_title('Largest singular value distribution vs Tracy-Widom approximation')
ax.legend()
ax.grid(True)
fig.show()


N=2, mean=1.6449659531156042
N=4, mean=2.3575373488636946
N=8, mean=2.92250459301298
N=16, mean=3.324841319111748
N=32, mean=3.574161182228103
N=64, mean=3.725208161269619
N=128, mean=3.8292628567352414
N=256, mean=3.8903456723464536


In [188]:
ss = largest_singular_values = largest_singular_value_distribution(256, 10)